<a href="https://colab.research.google.com/github/derek-shing/DS-Unit-4-Sprint-3-Neural-Networks/blob/master/module4-Hyperparameter-Tuning/LS_DS_434_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
##### Your Code Here #####
import keras
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy

Using TensorFlow backend.


In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/derek-shing/DS-Unit-4-Sprint-3-Neural-Networks/master/module4-Hyperparameter-Tuning/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [0]:
df.head()

In [6]:
df.shape

(7043, 21)

In [26]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,1,0,1,0,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,1,Electronic check,29.85,29.85,0
1,5575-GNVDE,Male,0,0,0,34,1,No,DSL,Yes,No,Yes,No,No,No,One year,0,Mailed check,56.95,1889.5,0
2,3668-QPYBK,Male,0,0,0,2,1,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,1,Mailed check,53.85,108.15,1
3,7795-CFOCW,Male,0,0,0,45,0,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,Female,0,0,0,2,1,No,Fiber optic,No,No,No,No,No,No,Month-to-month,1,Electronic check,70.70,151.65,1


In [0]:
pd.options.display.max_columns=100

In [25]:
binary_columns=['Partner','Dependents','PhoneService','PaperlessBilling','Churn']
Cat_columns=['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']
df[binary_columns]=df[binary_columns].replace({'Yes':1,'No':0})
for col in df[binary_columns].columns:
    print (col, df[col].unique())

Partner [1 0]
Dependents [0 1]
PhoneService [0 1]
PaperlessBilling [1 0]
Churn [0 1]


In [0]:
from sklearn.preprocessing import LabelEncoder
lb_encoder = LabelEncoder()
df['Contract']=lb_encoder.fit_transform(df['Contract'])

In [0]:
df =df.drop(columns ='customerID')

In [0]:
for col in df.select_dtypes(exclude='number').columns:
  df[col]=lb_encoder.fit_transform(df[col])

In [40]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,2505,0
1,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1466,0
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,157,1
3,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1400,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,925,1


In [0]:
X=df.drop(columns='Churn')
y=df['Churn']

In [42]:
X.shape, y.shape

((7043, 19), (7043,))

In [43]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [0]:
# Important Hyperparameters
inputs = X.shape[1]
epochs = 50
batch_size = 10

# Create Model
def create_model():
  model = Sequential()
  model.add(Dense(64, activation='relu', input_shape=(inputs,)))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
# Compile Model
  model.compile(loss ='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
  return model
  
# Fit Model
#model.fit(X, y, validation_split=0.33, epochs=epochs, batch_size=batch_size)

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
model = KerasClassifier(build_fn=create_model, verbose=1)
param_grid = {'batch_size': [80],
              'epochs': [20],
              #'activation':['relu','sigmoid']
             }

In [68]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/20
4695/4695 [==============================] - 3s 710us/step - loss: 0.5110 - acc: 0.7333
Epoch 2/20
4695/4695 [==============================] - 0s 26us/step - loss: 0.4399 - acc: 0.7911
Epoch 3/20
4695/4695 [==============================] - 0s 26us/step - loss: 0.4279 - acc: 0.7994
Epoch 4/20
4695/4695 [==============================] - 0s 26us/step - loss: 0.4234 - acc: 0.8009
Epoch 5/20
4695/4695 [==============================] - 0s 26us/step - loss: 0.4179 - acc: 0.8062
Epoch 6/20
4695/4695 [==============================] - 0s 26us/step - loss: 0.4153 - acc: 0.8081
Epoch 7/20
4695/4695 [==============================] - 0s 24us/step - loss: 0.4107 - acc: 0.8109
Epoch 8/20
4695/4695 [==============================] - 0s 27us/step - loss: 0.4079 - acc: 0.8128
Epoch 9/20
4695/4695 [==============================] - 0s 26us/step - loss: 0.4045 - acc: 0.8132
Epoch 10/20
4695/4695 [==============================] - 0s 25us/step - loss: 0.4023 - acc: 0.8138
Epoch 11/20
4695/4

In [69]:
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7942638051250891 using {'batch_size': 80, 'epochs': 20}
Means: 0.7942638051250891, Stdev: 0.006645692127553427 with: {'batch_size': 80, 'epochs': 20}


7043/7043 [==============================] - 0s 18us/step
acc: 87.68990487008377


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?